In [1]:
#Ed Costantini - costed12@gmail.com

In [ ]:
#Roto League Analysis 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#csv data taken from fangraphs auction calculator
hit, pit = 'fg_hitter','fg_pitcher'
fileHit   = hit + '.csv'
filePit = pit + '.csv'

In [3]:
#Read File
dfIn = pd.read_csv(fileHit)
dfIn.rename(columns={ dfIn.columns[0]: "PlayerName" },inplace=True)

dfInP = pd.read_csv(filePit)
dfInP.rename(columns={ dfInP.columns[0]: "PlayerName" },inplace=True)

In [4]:
#Player Floor
poolHitter=dfIn
poolHitter=poolHitter.dropna(subset=['ADP']).copy()

poolPitcher=dfInP
poolPitcher=poolPitcher.dropna(subset=['ADP']).copy()

In [5]:
#PrimaryPosition
poolHitter['PP']=poolHitter['POS']
positionOrder=['C','SS','2B','3B','OF','1B','SP','DH']
poolHitter=poolHitter.copy()
for pos in positionOrder:
    poolHitter.loc[poolHitter['PP'].str.contains(pos), 'PP'] = pos
    
poolPitcher['PP']=poolPitcher['POS']
positionOrder2=['SP','RP','DH']
poolPitcher=poolPitcher.copy()
for pos in positionOrder2:
    poolPitcher.loc[poolPitcher['PP'].str.contains(pos), 'PP'] = pos
    

In [6]:
#get hitter
def getHitter(playerName):
    return (poolHitter.loc[poolHitter['PlayerName']==playerName])

In [7]:
#get pitcher
def getPitcher(playerName):
    return (poolPitcher.loc[poolPitcher['PlayerName']==playerName])

In [8]:
#add hitter to team
def addHitter(playerNames,team):
    for i in range(len(playerNames)):
        name=playerNames[i]
        team=team.append(getHitter(name))
    return team

In [9]:
#add pitcher to team
def addPitcher(playerNames,team):
    for i in range(len(playerNames)):
        name=playerNames[i]
        team=team.append(getPitcher(name))
    return team

In [10]:
#remove player from team
def dropPlayer(playerName,team):
    target=team.loc[team['PlayerName']==playerName]
    team=team.drop(target)
    return team


In [11]:
#add cost to player
def addPlayerCost(playerName,cost):
    poolHitter.loc[poolHitter['PlayerName'] == playerName, 'Cost'] = cost
    

In [12]:
#add cost to pitcher
def addPitcherCost(playerName,cost):
    poolPitcher.loc[poolPitcher['PlayerName'] == playerName, 'Cost'] = cost
    

In [31]:
#create roster
myTeam=pd.DataFrame()
myRotation=pd.DataFrame()


In [15]:
availHitter=poolHitter[poolHitter['Cost'].isnull()]
availHitter.head()

,PlayerName,Team,POS,ADP,PA,mRBI,mR,mSB,mHR,mOBP,PTS,aPOS,Dollars,Cost,PP
0,Mike Trout,Angels,OF,1.1,635,9.8,13.3,6.5,9.4,18.5,57.5,11.2,69.8,NaN,OF
1,Giancarlo Stanton,Yankees,OF,8.9,584,15.8,10.1,-3.8,18.4,6.4,46.9,11.2,59.2,NaN,OF
2,Bryce Harper,Nationals,OF,8.5,589,7.3,7.5,0.1,7.5,14.1,36.5,11.2,48.7,NaN,OF
3,Paul Goldschmidt,Diamondbacks,1B,4.6,644,6.7,8.0,4.0,5.0,12.6,36.2,8.5,45.7,NaN,1B
4,Trea Turner,Nationals,SS,4.9,628,-3.3,7.1,23.7,-3.7,0.4,24.3,19.9,45.2,NaN,SS


In [16]:
availPitcher=poolPitcher[poolPitcher['Cost'].isnull()]
availPitcher.head()

,PlayerName,Team,POS,ADP,IP,mERA,mWHIP,mSO,mQS,mSVHLD,PTS,aPOS,Dollars,Cost,PP
0,Clayton Kershaw,Dodgers,SP,7.6,190,11.3,10.6,4.9,4.9,-2.2,29.5,8.6,39.0,NaN,SP
1,Chris Sale,Red Sox,SP,12.8,192,7.0,10.1,6.2,4.5,-2.2,25.5,8.6,35.1,NaN,SP
2,Max Scherzer,Nationals,SP,10.6,207,4.6,7.7,7.0,4.7,-2.2,21.8,8.6,31.4,NaN,SP
3,Noah Syndergaard,Mets,SP,34.1,182,7.0,6.9,5.0,3.8,-2.2,20.5,8.6,30.0,NaN,SP
4,Corey Kluber,Indians,SP,13.9,194,5.3,7.3,5.2,4.4,-2.2,20.1,8.6,29.7,NaN,SP


In [17]:
# #adjust hitter minimum price to 1
# #df2 = df [df['Adjusted'] > 0]
# optiHitter=availHitter.append(myTeam).copy()
# optiHitter.loc[optiHitter['Dollars'] < 1, 'Dollars'] = 1

# #all players who have a cost have their adjusted = their cost
# optiHitter.loc[optiHitter['Cost'] > 0, 'Dollars'] = optiHitter['Cost']
# optiHitter.head()

In [18]:
# #adjust pitcher minimum price to 1
# #df2 = df [df['Adjusted'] > 0]
# optiPitcher=availPitcher.append(myRotation).copy()
# optiPitcher.loc[optiPitcher['Dollars'] < 1, 'Dollars'] = 1

# #all players who have a cost have their adjusted = their cost
# optiPitcher.loc[optiPitcher['Cost'] > 0, 'Dollars'] = optiPitcher['Cost']
# optiPitcher.head()

In [19]:
import pulp

In [20]:
def hittingOptimizer(df):
    # initialize variables
    playerPositions = df['POS'].tolist()
    playerNames = df['PlayerName'].tolist()
    playerCosts = tuple(df['Dollars']) #using own projection prices
    playerPoints = tuple(df['PTS'])
    
    salaryCap = 260*.7-4
    numOfPlayers = range(len(playerCosts))
    
    # formulate linear programming problem
    problem = pulp.LpProblem("Optimal Lineup", pulp.LpMaximize)
    # create variable to represent each player
    playerInLineup = pulp.LpVariable.dicts("Players", [i for i in numOfPlayers], 0, 1, cat="Binary")
    # create objective: maximize sum of player points
    problem += pulp.lpSum(playerInLineup[i] * playerPoints[i] for i in numOfPlayers)
    
    # constraint: each player can only be chosen at most once
    for i in numOfPlayers:
        problem += pulp.lpSum(playerInLineup[i]) <= 1
    # constraints: sum of player costs must be less than or equal to the salary cap
    problem += sum(playerInLineup[i] * playerCosts[i] for i in numOfPlayers) <= salaryCap
    
    # constraint: teams must have 11 players
    #problem += pulp.lpSum(problem.variables()) == 9
    problem += pulp.lpSum(playerInLineup[i] for i in numOfPlayers) == 11 #of players in lineup
    
    #add keeper players from my team    
    #keep = tuple(df['Cost']>0)
    #problem += pulp.lpSum(playerInLineup[i] * keep[i] for i in numOfPlayers) == sum(keep)

    # positions
    # constraint: 1 = C
    #catchers = tuple(df['Pos'] == 'C')
    catchers = tuple(df['PP'].str.contains('C'))
    problem += pulp.lpSum(playerInLineup[i] * catchers[i] for i in numOfPlayers) >= 1
    
    # constraint: 1 <= number of 1b
    fb = tuple(df['PP'].str.contains('1B'))
    problem += pulp.lpSum(playerInLineup[i] * fb[i] for i in numOfPlayers) >= 1

    # constraint: 1 <= number of 2b
    sb = tuple(df['PP'].str.contains('2B'))
    problem += pulp.lpSum(sb[i] * playerInLineup[i] for i in numOfPlayers) >= 1

    # constraint: 1 <= number of 3b
    tb = tuple(df['PP'].str.contains('3B'))
    problem += pulp.lpSum(tb[i] * playerInLineup[i] for i in numOfPlayers) >= 1

    # constraint: 1 <= number of SS
    ss = tuple(df['PP'].str.contains('SS'))
    problem += pulp.lpSum(playerInLineup[i] * ss[i] for i in numOfPlayers) >= 1

    # constraint: 4 <= number of OF
    of = tuple(df['PP'].str.contains('OF'))
    problem += pulp.lpSum(playerInLineup[i] * of[i] for i in numOfPlayers) >= 4
    
    dh = tuple(df['PP']==('DH'))
    problem += pulp.lpSum(playerInLineup[i] * dh[i] for i in numOfPlayers) == 0  
     
    # output the calculated predicted lineup
    cost = 0
    points = 0

    predicted = list()
    if problem.solve() == 1:
        print ("Current Optimal Lineup")
        for pos in range(len(numOfPlayers)):
            if playerInLineup[pos].value() == 1:
                name = playerNames[pos]
                predicted.append(name)
                cost += playerCosts[pos]
                points += playerPoints[pos]
                
               
                print ('%25s, Position = %2s, Price = %5.f, Points = %3.2f' \
                      % (playerNames[pos], playerPositions[pos], playerCosts[pos],
                         playerPoints[pos]))
        print ('\nTotal Team Cost: %5d\nTotal Team Points: %5.2f' % (cost, points))
        
    else:
        print ('Error finding solution')
        
    return(predicted,cost,points)

In [21]:
def hitterValueOptimizer(df,hitterName,pointsH):
    # initialize variables
    df2=df.copy()
    df2.loc[df2['PlayerName'] == hitterName, 'Dollars'] = 0
    playerPositions = df2['POS'].tolist()
    playerNames = df2['PlayerName'].tolist()
    playerCosts = tuple(df2['Dollars'])
    playerPoints = tuple(df2['PTS'])
    targetPrice = 0
    
    salaryCap = 260*.7-4
    numOfPlayers = range(len(playerCosts))
    
    # formulate linear programming problemB
    problemB = pulp.LpProblem("Optimal Player Value", pulp.LpMaximize)
    # create variable to represent each player
    playerInLineup = pulp.LpVariable.dicts("Players", [i for i in numOfPlayers], 0, 1, cat="Binary")
    targetCost=pulp.LpVariable("TC")
        
    # create objective: maximize player 'adjusted' while including him in optimal lineup
    problemB+=targetCost
    
    #constraint: our player needs to be in the lineup
    target = tuple(df2['PlayerName']==hitterName)
    problemB += pulp.lpSum(playerInLineup[i] * target[i] for i in numOfPlayers) == 1
    
    #constraint: points needs to be >= last value
    problemB += pulp.lpSum(playerInLineup[i] * playerPoints[i] for i in numOfPlayers) >= pointsH
    
    #constraint: each player can only be chosen at most once
    for i in numOfPlayers:
        problemB += pulp.lpSum(playerInLineup[i]) <= 1
        
    # constraints: sum of player costs must be less than or equal to the salary cap
    problemB += pulp.lpSum(playerInLineup[i] * playerCosts[i] for i in numOfPlayers)+targetCost <= salaryCap#-targetCost
    
    # constraint: teams must have 11 players
    problemB += pulp.lpSum(playerInLineup[i] for i in numOfPlayers) == 11 #of players in lineup
    
    
    #add players from my team    
    keep = tuple(df2['Cost']>0)
    problemB += pulp.lpSum(playerInLineup[i] * keep[i] for i in numOfPlayers) == sum(keep)#of keepers

    # positions
    # constraint: 1 = C
    #catchers = tuple(df2['Pos'] == 'C')
    catchers = tuple(df2['PP'].str.contains('C'))
    problemB += pulp.lpSum(playerInLineup[i] * catchers[i] for i in numOfPlayers) >= 1

    # constraint: 1 <= number of 1b
    fb = tuple(df2['PP'].str.contains('1B'))
    problemB += pulp.lpSum(playerInLineup[i] * fb[i] for i in numOfPlayers) >= 1

    # constraint: 1 <= number of 2b
    sb = tuple(df2['PP'].str.contains('2B'))
    problemB += pulp.lpSum(sb[i] * playerInLineup[i] for i in numOfPlayers) >= 1

    # constraint: 1 <= number of 3b
    tb = tuple(df2['PP'].str.contains('3B'))
    problemB += pulp.lpSum(tb[i] * playerInLineup[i] for i in numOfPlayers) >= 1

    # constraint: 1 <= number of SS
    ss = tuple(df2['PP'].str.contains('SS'))
    problemB += pulp.lpSum(playerInLineup[i] * ss[i] for i in numOfPlayers) >= 1
    
    # constraint: 4 <= number of OF
    of = tuple(df2['PP'].str.contains('OF'))
    problemB += pulp.lpSum(playerInLineup[i] * of[i] for i in numOfPlayers) >= 4
    
    #lazy dh fix
    dh = tuple(df2['PP']==('DH'))
    problemB += pulp.lpSum(playerInLineup[i] * dh[i] for i in numOfPlayers) == 0  
     
    # output the calculated predicted lineup
    cost = 0
    points = 0
    
    predicted = list()
    if problemB.solve() == 1:
#        print ("Hypothetical Lineup")
        for pos in range(len(numOfPlayers)):
            if playerInLineup[pos].value() == 1:
                name = playerNames[pos]
                predicted.append(name)
                cost += playerCosts[pos]
                points += playerPoints[pos]
                
        print (hitterName,targetCost.value())
        
    else:
        print ('Error finding solution')
        
    return (predicted,cost,points,targetCost.value())

In [22]:
def pitchingOptimizer(df):
    # initialize variables
    playerPositions = df['POS'].tolist()
    playerNames = df['PlayerName'].tolist()
    playerCosts = tuple(df['Dollars']) #using own projection prices
    playerPoints = tuple(df['PTS'])
    
    salaryCap = 260*.3
    numOfPlayers = range(len(playerCosts))
    
    # formulate linear programming problem
    problem = pulp.LpProblem("Optimal Lineup", pulp.LpMaximize)
    # create variable to represent each player
    playerInLineup = pulp.LpVariable.dicts("Players", [i for i in numOfPlayers], 0, 1, cat="Binary")
    # create objective: maximize sum of player points
    problem += pulp.lpSum(playerInLineup[i] * playerPoints[i] for i in numOfPlayers)
    
    # constraint: each player can only be chosen at most once
    for i in numOfPlayers:
        problem += pulp.lpSum(playerInLineup[i]) <= 1
    # constraints: sum of player costs must be less than or equal to the salary cap
    problem += sum(playerInLineup[i] * playerCosts[i] for i in numOfPlayers) <= salaryCap
    
    # constraint: teams must have 11 players
    problem += pulp.lpSum(playerInLineup[i] for i in numOfPlayers) == 10 #of players in lineup
    
    #add players from my team    
    keep = tuple(df['Cost']>0)
    problem += pulp.lpSum(playerInLineup[i] * keep[i] for i in numOfPlayers) == sum(keep)

    # positions
    # constraint: number of SP
    sp = tuple(df['PP'].str.contains('SP'))
    problem += pulp.lpSum(playerInLineup[i] * sp[i] for i in numOfPlayers) >= 6
    
    # constraint: number of RP
    rp = tuple(df['PP'].str.contains('RP'))
    problem += pulp.lpSum(playerInLineup[i] * rp[i] for i in numOfPlayers) >= 4 
     
    # output the calculated predicted lineup
    cost = 0
    points = 0

    predicted = list()
    if problem.solve() == 1:
        print ("Current Optimal Rotation")
        for pos in range(len(numOfPlayers)):
            if playerInLineup[pos].value() == 1:
                name = playerNames[pos]
                predicted.append(name)
                cost += playerCosts[pos]
                points += playerPoints[pos]
                
               
                print ('%25s, Position = %2s, Price = %5.f, Points = %3.2f' \
                      % (playerNames[pos], playerPositions[pos], playerCosts[pos],
                         playerPoints[pos]))
        print ('\nTotal Team Cost: %5d\nTotal Team Points: %5.2f' % (cost, points))
        
    else:
        print ('Error finding solution')
        
    return(predicted,cost,points)

In [23]:
def pitcherValueOptimizer(df,pitcherName,pointsP):
    # initialize variables
    df2=df.copy()
    df2.loc[df2['PlayerName'] == pitcherName, 'Dollars'] = 0
    playerPositions = df2['POS'].tolist()
    playerNames = df2['PlayerName'].tolist()
    playerCosts = tuple(df2['Dollars'])
    playerPoints = tuple(df2['PTS'])
    targetPrice = 0
    
    salaryCap = 260*.3
    numOfPlayers = range(len(playerCosts))
    
    # formulate linear programming problemB
    problemB = pulp.LpProblem("Optimal Player Value", pulp.LpMaximize)
    # create variable to represent each player
    playerInLineup = pulp.LpVariable.dicts("Players", [i for i in numOfPlayers], 0, 1, cat="Binary")
    targetCost=pulp.LpVariable("TC")
        
    # create objective: maximize player 'adjusted' while including him in optimal lineup
    problemB+=targetCost
    
    #constraint: our player needs to be in the lineup
    target = tuple(df2['PlayerName']==pitcherName)
    problemB += pulp.lpSum(playerInLineup[i] * target[i] for i in numOfPlayers) == 1
    
    #constraint: points needs to be >= last value
    problemB += pulp.lpSum(playerInLineup[i] * playerPoints[i] for i in numOfPlayers) >= pointsP
    
    #constraint: each player can only be chosen at most once
    for i in numOfPlayers:
        problemB += pulp.lpSum(playerInLineup[i]) <= 1
        
    # constraints: sum of player costs must be less than or equal to the salary cap
    problemB += pulp.lpSum(playerInLineup[i] * playerCosts[i] for i in numOfPlayers)+targetCost <= salaryCap#-targetCost
    
    # constraint: teams must have 11 players
    badCode = tuple(df2['IP'].notnull())
    problemB += pulp.lpSum(playerInLineup[i] * badCode[i] for i in numOfPlayers) == 10 #of players in lineup
    
    
    #add players from my team    
    keep = tuple(df2['Cost']>0)
    problemB += pulp.lpSum(playerInLineup[i] * keep[i] for i in numOfPlayers) == sum(keep)#of keepers

    # positions
    # constraint: number of SP
    sp = tuple(df['PP'].str.contains('SP'))
    problemB += pulp.lpSum(playerInLineup[i] * sp[i] for i in numOfPlayers) >= 6
    
    # constraint: number of RP
    rp = tuple(df['PP'].str.contains('RP'))
    problemB += pulp.lpSum(playerInLineup[i] * rp[i] for i in numOfPlayers) >= 4  
     
    # output the calculated predicted lineup
    cost = 0
    points = 0
    
    predicted = list()
    if problemB.solve() == 1:
#        print ("Hypothetical Lineup")
        for pos in range(len(numOfPlayers)):
            if playerInLineup[pos].value() == 1:
                name = playerNames[pos]
                predicted.append(name)
                cost += playerCosts[pos]
                points += playerPoints[pos]
                
        print (pitcherName,targetCost.value())
        
    else:
        print ('Error finding solution')
        
    return (predicted,cost,points,targetCost.value())

In [28]:
#reset after draft
def resetDraft():
    myTeam=pd.DataFrame()
    poolHitter=dfIn
    poolHitter=poolHitter.dropna(subset=['ADP']).copy()
    poolHitter['PP']=poolHitter['POS']
    positionOrder=['C','SS','2B','3B','OF','1B','SP','DH']
    poolHitter=poolHitter.copy()
    for pos in positionOrder:
        poolHitter.loc[poolHitter['PP'].str.contains(pos), 'PP'] = pos


    myRotation=pd.DataFrame()
    poolPitcher=dfInP
    poolPitcher=poolPitcher.dropna(subset=['ADP']).copy()
    poolPitcher['PP']=poolPitcher['POS']
    positionOrder2=['SP','RP','DH']
    poolPitcher=poolPitcher.copy()
    for pos in positionOrder2:
        poolPitcher.loc[poolPitcher['PP'].str.contains(pos), 'PP'] = pos

    hitter=True

In [35]:
#simulate live draft with integer number of picks
def liveDraft(picks): 
    resetDraft()
    myTeam=pd.DataFrame()
    myRotation=pd.DataFrame()
    hitter=True
    for i in range(0,picks):
        availHitter=poolHitter[poolHitter['Cost'].isnull()]
        availPitcher=poolPitcher[poolPitcher['Cost'].isnull()]

        #adjust minimum price to 1
        optiHitter=availHitter.append(myTeam).copy()
        optiHitter.loc[optiHitter['Dollars'] < 1, 'Dollars'] = 1

        #all players who have a cost have their dollars = their cost
        optiHitter.loc[optiHitter['Cost'] > 0, 'Dollars'] = optiHitter['Cost']

        #adjust minimum price to 1
        optiPitcher=availPitcher.append(myRotation).copy()
        optiPitcher.loc[optiPitcher['Dollars'] < 1, 'Dollars'] = 1

        #all players who have a cost have their adjusted = their cost
        optiPitcher.loc[optiPitcher['Cost'] > 0, 'Dollars'] = optiPitcher['Cost']

        #lineup optimization
        if(i==0):
            #predictedLineup, costH, pointsH  hittingOptimizer(optiHitter)
            predictedRotation, costP, pointsP = pitchingOptimizer(optiPitcher)
        if(hitter):
            predictedLineup, costH, pointsH = hittingOptimizer(optiHitter)
        else:
            predictedRotation, costP, pointsP = pitchingOptimizer(optiPitcher)


        #input nominated player
        nomPlayerName = input("Please enter nominated player name: ")
        #check if player is hitter or pitcher
        isPitcher = input("Is this player a pitcher? y/n")
        if(isPitcher=='y'):
                hitter=False
        if(hitter):
            #determine optimal cost
            predictedLineup2, costH2, pointsH2,targetCost = hitterValueOptimizer(optiHitter,nomPlayerName,pointsH)
            print(getHitter(nomPlayerName))

            #update player cost
            auctionSold = int(input("Enter sold price: "))
            addPlayerCost(nomPlayerName,auctionSold)

            #did you draft this player
            youDrafted = input("Did you draft this player? y or n")
            if(youDrafted=='y'):
                myTeam=addHitter([nomPlayerName], myTeam)
        else:
            predictedRotation2, costP2, pointsP2,targetCost = pitcherValueOptimizer(optiPitcher,nomPlayerName,pointsP)
            print(getPitcher(nomPlayerName))

            #update player cost
            auctionSold = int(input("Enter sold price: "))
            addPitcherCost(nomPlayerName,auctionSold)

            #did you draft this player
            youDrafted = input("Did you draft this player? y or n")
            if(youDrafted=='y'):
                myRotation=addPitcher([nomPlayerName], myRotation)  
    

In [36]:
liveDraft(3)

Current Optimal Rotation
          Clayton Kershaw, Position = SP, Price =    39, Points = 29.50
          Aroldis Chapman, Position = RP, Price =    12, Points = 5.80
            Felipe Rivero, Position = RP, Price =     9, Points = 3.50
           Jeurys Familia, Position = RP, Price =     4, Points = -1.50
             Jake Arrieta, Position = SP, Price =     4, Points = -5.70
              Brett Cecil, Position = RP, Price =     4, Points = -2.00
            Drew Pomeranz, Position = SP, Price =     2, Points = -7.40
                J.A. Happ, Position = SP, Price =     2, Points = -7.50
            Brock Stewart, Position = SP, Price =     1, Points = -8.20
               Jake Faria, Position = SP, Price =     1, Points = -8.30

Total Team Cost:    78
Total Team Points: -1.80
Current Optimal Lineup
             Bryce Harper, Position = OF, Price =    49, Points = 36.50
             Gary Sanchez, Position =  C, Price =    34, Points = -0.50
          Freddie Freeman, Position = 1B,